##### <h1 align=center><font size=5>SAM Python Capstone Final Project</font></h1>

## Description

<p>The final project is an automatic grading system.  The goal is open each of the files included in
the compressed file</p>

### Instructions
1.  [Create a class roster by parsing the file names](#0)<br>
2.  [Create a student exam GPA](#2)<br>
3.  [Create a summary report](#4) <br>
4.  [Create an input screen for weighted average](#6) <br> 
5.  [Apply weighted average- create report](#8) <br>  

### Class roster<a id="0"></a>
#### 10 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER.TXT'
#### In should contain an unduplicate list of class names
#### 1. Lastname, Firstname
#### 2. Lastname,Firstname
#### NOTE: The names should be in alphabetical order by last name, first name and in proper case
#### You must parse the file names to get the student names:  CAPSTONE_lastname_firstname_QUIZ.......txt

In [ ]:
# Give partial credit on multiple answer questions if some are correct
partial_credit = True

# ------- CREATING CLASS ROSTER ------- 10 Pts ------------------------------------------------------------
roster = 'python_pti_capstone_class_roster.txt'

import os
file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

# List of student names sorted by last name
student_roster = sorted(set([
    # parsed 
    ' '.join(student.lower().replace('capstone_','').split("_")[:2]) for student in file_names
    ]))

# Write them to the specified filename and close the file
file = open(roster, 'w')
[file.write(student.split()[0].capitalize()+ ' ' + student.split()[1].capitalize() +'\n') for student in student_roster if student != 'final exam' and student != 'quiz key']
file.close()
#----------------------------------------------------------------------------------------------------------

In [ ]:
# Create function that takes in the name of a file and returns the grade
from pathlib import Path
def grade_assignment(assignment_name='CAPSTONE_davis_FRANK_QUIZ_1_LESSON_1.TXT'):
    '''
    Takes in a file name and returns the score by comparing the exam against the key line by line.
    Paritial credit is possible for multi-answer questions if the partial_credit is set to True.
    '''
    file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

    # List of student names sorted by last name
    keys = sorted(set([
        key for key in file_names if 'key' in key.lower()
    ]))

    # Use the last character+file_extension to match the key to the assignment provided
    assignment_key = [key for key in keys if key.split('_')[-1].lower() == assignment_name.split('_')[-1].lower()]
    if len(assignment_key) == 0:
        assignment_key = 'CAPSTONE_FINAL_EXAM_KEY.TXT'
    else:
        assignment_key = assignment_key[0]

    #------------------- Could be a seperate function -----------------
    # read the students answers
    student_answers = open(Path('./PYTHON_PTI_CAPSTONE_PROJECT_FILES/' + assignment_name), 'r')
    student_answers = [answer.strip().replace(',','').replace('-','') for answer in student_answers]
    # read the key answers
    key_answers = open(Path('./PYTHON_PTI_CAPSTONE_PROJECT_FILES/' + assignment_key), 'r')
    key_answers = [answer.strip().replace(',','').replace('-','') for answer in key_answers]
    #-------------------------------------------------------------------

    # remember how many questions/points on the assignment
    number_of_questions = len(key_answers)
    grade = 0 # just initializing
    # give the student a point for every question they got correct
    for question in range(number_of_questions):
        question_value = 0
        # Avoiding index errors by caused by mismatching lengths
        try:
            question_points_possible = len(key_answers[question])-1
            question_points_earned = 0

            if partial_credit == True:
                for answer in range(question_points_possible):
                    # If the student answer matches the key, give give a point
                    if key_answers[question][answer+1] == student_answers[question][answer+1]:
                        question_points_earned += 1
                    question_value += 1
            else:  
                # print(key_answers[question],student_answers[question])
                if student_answers[question] == key_answers[question]:
                    grade += 1
        except:
            # didn't answer a question... that's embarrassing
            pass
        if partial_credit == True:
            question_score = question_points_earned / question_value
        grade += question_score
        
    # return their grade EXAMPLE 90.0
    return (grade/number_of_questions)*100

grade_assignment()

In [ ]:
# BUILDING GRADE DICTIONARY--Temporary
import pickle
student_dictionary = {}

# student names in lower case
students_lower = []
file = open(roster,'r')
for line in file:
    students_lower.append(line.strip())

# get list of all assignements (sort assignments by name for cleanliness)
file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')
assignments = sorted([assignment for assignment in file_names])
# Get all assignments per student and create the dictionary
for student in students_lower:   
    student_dictionary[student] = {
        # get assignment grade during dict construction
        item : int(grade_assignment(item)) for item in assignments if '_'.join(student.lower().split()) in item.lower()
    }

    # update the students dictionary item with... the average of their scores
    student_dictionary[student]['mean'] = \
    int(sum([grade for _ ,grade in student_dictionary[student].items()])/6)

# save the student_dictionary via Pickle
with open('student_data.dat', 'wb') as file:
    pickle.dump(student_dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)

### Class roster with mean test scores<a id="2"></a>
#### 30 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER_GPA.TXT'
#### In should contain an unduplicate list of student names their graded scores and their mean
#### 1. Lastname, Firstname - Test Score 1, Test Score 2, ... Final Exam Score, Mean
#### 2. Lastname, Firstname - Test Score 1, Test Score 2, ... Final Exam Score, Mean
#### NOTE: The names should be in alphabetical order by last name, first name and in proper case
#### Use the file 'CAPSTONE_QUIZ_KEY_LESSON_1' through 5 to score the exams and calculate the mean

In [ ]:
filename = 'PYTHON_PTI_CAPSTONE_CLASS_ROSTER_GPA.TXT'
# -----CLass Roster With Mean Scores----------30 pts------------------------------

student_exam_grades = []
for student in student_dictionary.items():
    # put the students grades into a line
    line = [str(grade) for _,grade in student[1].items()]
    # arrange the final grade to appear before the mean, and insert the students name
    line.insert(-1,line.pop(0)), line.insert(0,student[0])
    student_exam_grades.append(line)

# Storing with Pickle
with open('student_exam_grades.dat', 'wb') as file:
    pickle.dump(student_exam_grades, file, protocol=pickle.HIGHEST_PROTOCOL)

# Storing as text file
file = open(filename, 'w')
for line in student_exam_grades:
    # Write the line to the specified file
    file.write(' '.join(line)+'\n')
file.close()
#---------------------------------------------------------------------------------

In [ ]:
import statistics
# Now that the students names and their grades are in a convenient object
#    A single dictionary can be built containing all the exam and student statistics required
def data_constructor(weight1=0,weight2=0):
     with open('student_exam_grades.dat', 'rb') as file:
          student_exam_grades = pickle.load(file)

     # Will need all the file names 
     file_names = os.listdir('./PYTHON_PTI_CAPSTONE_PROJECT_FILES')

     # Exam names
     exam_key_names = sorted([name for name in file_names if 'key' in name.lower()])
     exam_key_names = exam_key_names + [exam_key_names.pop(0)]

     #Initializing now to add to them dynamically later
     class_data = {'exams':{},'students':{}}

     # Exclude counting the student name and their average score
     for index in range(len(student_exam_grades[0][1:-1])):

          # Gather student scores on the assignment
          grades = [int(student_exam_grades[exam][index+1]) for exam in range(len(student_exam_grades)-1)]

          # create a item with the key being the assignment name, and fill it will required statistics
          class_data['exams'][exam_key_names[index]] = {
               'class_score_mean' : statistics.mean(grades),
               'class_score_median' : statistics.median(grades),
               'class_score_mode'  : statistics.mode(grades),
               'standard_deviation' : statistics.stdev(grades),
               'times_taken' : len(student_exam_grades),
               'weight' : weight2 if 'final' in exam_key_names[index].lower() else weight1,
          },

     # for every student
     for student in range(len(student_exam_grades)):
          class_data['students'][student_exam_grades[student][0]] = {}
          for name in file_names:
               # if the test belongs to the student
               if student_exam_grades[student][0].replace(' ','_').lower() in name.lower():
                    # sore the student as a key,value      EXAMPLE:  student_name:{ assignment1:score, assignment2:score, etc.. }
                    class_data['students'][student_exam_grades[student][0]][name] = \
                    student_exam_grades[student][len(class_data['students'][student_exam_grades[student][0]])+1]

                    'score'

     # Save the constructed data object
     with open('class_data.dat', 'wb') as file:
          pickle.dump(class_data, file, protocol=pickle.HIGHEST_PROTOCOL)
     return class_data   
data_constructor()

### Class summary report<a id="4"></a>
#### 15 points
#### Create a file called 'PYTHON_PTI_CAPSTONE_SUMMARY REPORT.TXT'
#### It should contain the following
#### 1. There were X students
#### 2. Exam 1 - mean, standard deviation
#### 3. Exam 2 - mean, standard deviation
#### 4. Exam 3 - mean, standard deviation
#### 5. Exam 4 - mean, standard deviation
#### 6. Exam 5 - mean, standard deviation
#### 7. Final exam - mean, standard deviation

In [ ]:
filename = 'PYTHON_PTI_CAPSTONE_SUMMARY REPORT.TXT'

with open('class_data.dat', 'rb') as file:
     student_exam_grades = pickle.load(file)

file = open('PYTHON_PTI_CAPSTONE_SUMMARY REPORT.TXT', 'w')
for index, item in enumerate(student_exam_grades['exams']):
     file.write(f'''{index+1}. {item} - mean: {student_exam_grades['exams'][item][0]['class_score_mean']}, standard deviation: {student_exam_grades['exams'][item][0]['standard_deviation']}\n''')
file.close()

### Class summary report<a id="6"></a>
#### 15 points
#### Create an input screen which saves the following weights: Exam 1 through 5 are 15 points and final is 25 points
#### Exam #
#### Weight
#### NOTE: BONUS - Pickle the data as a dictionary instead of creating a file 'PYTHON_PTI_CAPSTONE_WEIGHTS.TXT'

In [ ]:
# The weights were initialized with a value of zero, when calling the function
#   the dictionary will be reconstructed with provided weight values and re-pickled

data_constructor(input('Enter weight for Exams 1-5'),input('Enter weight for Final'))

### Final grade for roster<a id="8"></a>
#### 30 points
#### List every student from highest to lowest final grade without weights PYTHON_PTI_CAPSTONE_GRADES.TXT
#### List every student from highest to lowest final grade with weights  PYTHON_PTI_CAPSTONE_WEIGHTED_GRADES.TXT
#### Use must open and use the PYTHON_PTI_CAPSTONE_WEIGHTS.TXT File created above
#### 1. Last name, first name - Final grade
#### 2. Last name, first name - Final grade
#### Add a summary
#### Mean, Mode, Median, Standard deviation
#### How many students are within 1 standard deviation of the mean
#### How many students are within 2 standard deviations of the mean
#### How many students are within 3 standard deviations of the mean
#### Is this a normal distribution?

In [ ]:
# The burnout is real.........